# Sketching Methods for Analysis of Matrices and Data (Fall 2019) - Final Project
## Comparison of Least-Square Solvers - Elad Eatah

### TODO: Update link to the notebook in the repo!
<a href="????"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
[![MIT License](https://img.shields.io/apm/l/atomic-design-ui.svg?)](https://github.com/tterb/atomic-design-ui/blob/master/LICENSEs)

Complete description of this project is avilable in [this repo](https://github.com/RedCrow9564/SketchingMethodsInDataAnalysis-Final-Project/tree/develop).

## Getting Started
First run the nodes under "Infrastructure" and "Compared Algorithms" one by one.
Second, run the first two nodes under "Main Components".

### Running Unit-Tests
You can then run the nodes under "Unit-Tests" one by one. The results of the last node are the results of these Unit-Tests.

### Performing an experiment
This is done by running the "Main" node. The configuration of the experiment
can be set in the "config" function in the "Main" node.

##Infrastructure

In [88]:
#@title Dependencies installations
!pip install intel-numpy 
!pip install intel-scipy 
!pip install intel-scikit-learn
!pip install nptyping
!pip install sacred
!pip install Cython
from IPython.display import clear_output
from nptyping import Array
from typing import List, Dict, Callable, ClassVar, Union, Iterator
import numpy as np
import warnings
import pandas as pd
import psutil
import os
import inspect
from multiprocessing import Pool
from time import time, perf_counter
from sacred import Experiment
#warnings.filterwarnings("error")  # Uncomment to see warnings as errors.
%load_ext Cython
cpu_count: int = psutil.cpu_count(logical=True)
clear_output()
print(f'Total CPUs: {cpu_count}')
print("Installation is done!")

Total CPUs: 4
Installation is done!


In [89]:
#@title Utils
ex = Experiment(name="Initializing project", interactive=True)


# Naming data types for type hinting.
Number = Union[int, float]
Scalar = Union[Number, Array[float, 1, 1], Array[int, 1, 1]]
RowVector = Union[List[Scalar], Array[float, 1, ...], Array[int, 1, ...], Scalar]
ColumnVector = Union[List[Scalar], Array[float, ..., 1], Array[int, ..., 1], Scalar]
Vector = Union[RowVector, ColumnVector]
Matrix = Union[List[Vector], Array[float], Array[int], Vector, Scalar]
StaticField = ClassVar


class _MetaEnum(type):
    """
    A private meta-class which given any :class:`BaseEnum` object to be an iterable.
    This can be used for iterating all possible values of this enum. Should not be used explicitly.
    """
    def __iter__(self) -> Iterator:
        """
        This method gives any BaseEnum the ability of iterating over all the enum's values.

        Returns:
            An iterator for the collection of all the enum's values.

        """
        # noinspection PyUnresolvedReferences
        return self.enum_iter()

    def __contains__(self, item) -> bool:
        """
        This method give any BaseEnum the ability to test if a given item is a possible value for this enum class.

        Returns:
            A flag which indicates if 'item' is a possible value for this enum class.

        """
        # noinspection PyUnresolvedReferences
        return self.enum_contains(item)


class BaseEnum(metaclass=_MetaEnum):
    """
    A basic interface for all enum classes. Should be sub-classed in eny enum, i.e ``class ExperimentType(BaseEnum)``
    """

    @classmethod
    def enum_iter(cls) -> Iterator:
        """
        This method gives any BaseEnum the ability of iterating over all the enum's values.

        Returns:
            An iterator for the collection of all the enum's values.

        """
        return iter(cls.get_all_values())

    @classmethod
    def enum_contains(cls, item) -> bool:
        """
        This method give any BaseEnum the ability to test if a given item is a possible value for this enum class.

        Returns:
            A flag which indicates if 'item' is a possible value for this enum class.

        """
        return item in cls.get_all_values()

    @classmethod
    def get_all_values(cls) -> List:
        """
        A method which fetches all possible values of an enum. Used for iterating over an enum.

        Returns:
            A list of all possible enum's values.

        """
        all_attributes: List = inspect.getmembers(cls, lambda a: not inspect.ismethod(a))
        all_attributes = [value for name, value in all_attributes if not (name.startswith('__') or name.endswith('__'))]
        return all_attributes


def create_factory(possibilities_dict: Dict[str, Callable], are_methods: bool = False) -> Callable:
    """
    A generic method for creating factories for the entire project.

    Args:
        possibilities_dict(Dict[str, Callable]): The dictionary which maps object types (as strings!) and returns the
            relevant class constructors.
        are_methods(bool): A flag, true if the factory output are methods, rather than objects. Defaults to False

    Returns:
         The factory function for the given classes/methods mapping.

    """
    def factory_func(requested_object_type: str):  # Inner function!
        if requested_object_type not in possibilities_dict:
            raise ValueError("Object type {0} is NOT supported".format(requested_object_type))
        else:
            if are_methods:
                return possibilities_dict[requested_object_type]
            else:
                return possibilities_dict[requested_object_type]()

    return factory_func


def compute_parallel(func: Callable, arguments) -> List:
    """
    A method which computes a given method for given arguments in parallel and returns its result in a list.

    Args:
        func(Callable): A function whose computation will be performed in parallel.
        arguments:  A collection for given inputs to the input method 'func'.

    Returns:
        A list for all outputs of this method for all given input arguments.

    """
    pool = Pool()
    results: List = pool.starmap(func, arguments)
    pool.close()
    return results


def compute_serial(func: Callable, arguments) -> List:
    """
        A method which computes a given method for given arguments in parallel and returns its result in a list.

        Args:
            func(Callable): A function whose computation will be performed in parallel.
            arguments:  A collection for given inputs to the input method 'func'.

        Returns:
            A list for all outputs of this method for all given input arguments.

        """
    results = list()
    for argument in arguments:
        results.append(func(*argument))
    return results


def measure_time(method: Callable) -> Callable:
    """
    A method which receives a method and returns the same method, while including run-time measure
    output for the given method, in seconds.

    Args:
        method(Callable): A method whose run-time we are interested in measuring.

    Returns:
        A function which does exactly the same, with an additional run-time output value in seconds.

    """
    def timed(*args, **kw):
        ts = perf_counter()
        result = method(*args, **kw)
        te = perf_counter()
        duration_in_seconds: float = te - ts
        return (result, duration_in_seconds)
    timed.__name__ = method.__name__ + " with time measure"
    return timed


def is_empty(collection: List) -> bool:
    return len(collection) == 0


class DataLog:
    """
    A class for log-management objects. See the following example for creating it: ``DataLog(["Column 1", "Column 2"])``
    """
    def __init__(self, log_fields: List):
        """
        This methods initializes an empty log.

        Args:
            log_fields(List) - A list of column names for this log.

        """
        self._data: Dict = dict()
        self._log_fields: List = log_fields
        for log_field in log_fields:
            self._data[log_field] = list()

    def append(self, data_type: str, value: Vector) -> None:
        """
        This methods appends given data to the given column inside the log.
        Example of usage:``log.append(DataFields.DataSize, 20)``

        Args:
            data_type(LogFields): The column name in which the input data in inserted to.
            value(Scalar): The value to insert to the log.

        """
        self._data[data_type] += value

    def append_dict(self, data_dict: Dict) -> None:
        for log_field, data_value in data_dict.items():
            self.append(log_field, data_value)

    def save_log(self, log_file_name: str, results_folder_path: str) -> None:
        """
        This method saves the log to a file, with the input name, in the input folder path.

        Args:
            log_file_name(str): The name for this log file.
            results_folder_path(str): The path in which this log will be saved.

        """
        df = pd.DataFrame(self._data, columns=self._log_fields)
        df.to_csv(os.path.join(results_folder_path, log_file_name + ".csv"), sep=",", float_format="%.2E", index=False)
        ex.info["Experiment Log"] = self._data

print("Utils loaded successfully!")

Utils loaded successfully!


In [90]:
#@title Enums
# -*- coding: utf-8 -*-
"""
enums.py - All enums section
============================

This module contains all possible enums of this project. Most of them are used by the configuration section in
:mod:`main`. An example for using enum: ``ExperimentType.RunTimeExperiment``

"""

class LogFields(BaseEnum):
    """
    The enum class of fields within experiments logs.
    """
    DataSize: str = "Data size"
    Coefficients: str = "Coefficients"
    Residuals: str = "Residuals" #  2-norm of errors of estimation for the given data.
    DurationInSeconds: str = "Duration in seconds"
    AtTimesErrors: str = "A transpose times Errors"
    AlphasCount: str = "Alphas count"


class DatabaseType(BaseEnum):
    """
    The enum class of dataset type to use in an experiment.
    """
    Synthetic: str = "Synthetic"  # A random data

    # 3D road network with highly accurate elevation information (+-20cm) from Denmark,
    # used in eco-routing and fuel/Co2-estimation routing algorithms.
    # See https://archive.ics.uci.edu/ml/datasets/3D+Road+Network+(North+Jutland,+Denmark)
    ThreeDRoadNetwork: str = "3D Road Network"

    # It includes homes sold between May 2014 and May 2015. See https://www.kaggle.com/harlfoxem/housesalesprediction
    HouseSalesInKingCounty: str = "House Sales in King County, USA"

    # Measurements of electric power consumption in one household with a one-minute sampling rate over a period of
    # almost 4 years. This archive contains 2075259 measurements gathered in a house located in Sceaux
    # #(7km of Paris, France) between December 2006 and November 2010 (47 months).
    # See https://archive.ics.uci.edu/ml/datasets/Individual+household+electric+power+consumption.
    IndividualHouseholdElectricPowerConsumption: str = "Individual household electric power consumption"


class ExperimentType(BaseEnum):
    """
    The enum class of experiment types.
    """
    RunTimeExperiment: str = "Run-Time Experiment"
    AccuracyExperiment: str = "Accuracy Experiment"
    NumberOfAlphasExperiment: str = "Number of Alphas Experiment"


class AlgorithmsType(BaseEnum):
    """
    The enum class of algorithms type to be compared in an experiment.
    """
    LinearRegression: str = "Linear Regression"
    LassoRegression: str = "Lasso Regression"
    RidgeRegression: str = "Ridge Regression"
    ElasticNetRegression: str = "ElasticNet Regression"


class LinearRegressionMethods(BaseEnum):
    """
    The enum class of possible Linear-Regression solvers for experiments.
    """
    BoostedSVDSolver: str = "SVD solver with Caratheodory Coreset Booster"
    SVDBased: str = "Based on SVD"  # Numpy's standard linear-regression solver- lstsq, based on SVD decomposition.
    QRBased: str = "Based on QR"  # Solver based on QR decomposition.
    NormalEquationsBased: str = "Based on Normal-Equations"  # Solver based on solving the Normal-Equations.
    SketchAndCholesky: str = "Sketch + Cholesky"
    SketchAndInverse: str = "Sketch + Inverse"
    SketchPreconditioned: str = "Sketch Preconsitioning for LSQR"


class LassoRegressionMethods(BaseEnum):
    """
    The enum class of possible Lasso-Regression solvers for experiments.
    """
    SkLearnLassoRegression: str = "Scikit-Learn's LassoCV Method"  # Scikit-learn standard Lasso-Regression solver.

    # The solver of Scikit-Learn, boosted by Caratheodory coreset booster.
    BoostedLassoRegression: str = "LassoCV with Fast Caratheodory booster"


class RidgeRegressionMethods(BaseEnum):
    """
    The enum class of possible Ridge-Regression solvers for experiments.
    """
    SkLearnRidgeRegression: str = "Scikit-Learn's RidgeCV Method"  # Scikit-learn standard Ridge-Regression solver.

    # The solver of Scikit-Learn, boosted by Caratheodory coreset booster.
    BoostedRidgeRegression: str = "RidgeCV with Fast Caratheodory booster"


class ElasticNetRegressionMethods(BaseEnum):
    """
    The enum class of possible Elastic-Net-Regression solvers for experiments.
    """
    # Scikit-learn standard Elastic-Net-Regression solver.
    SkLearnElasticNetRegression: str = "Scikit-Learn's ElasticNetCV Method"

    # The solver of Scikit-Learn, boosted by Caratheodory coreset booster.
    BoostedElasticNetRegression: str = "ElasticNetCV with Fast Caratheodory booster"


class NumpyDistribution(BaseEnum):
    """
    The type of Numpy distribution used within an experiment. Used for experiment documentation only.
    """
    IntelDistribution: str = "Intel's Distribution"
    CPythonDistribution: str = "CPython's distribution"


print("Enums loaded successfully!")


Enums loaded successfully!


#Compared Algorithms

In [0]:
#@title base-least-square-solver

class BaseSolver(object):
    def __init__(self, data_features: Matrix, output_samples: ColumnVector, n_alphas: int,
                 cross_validation_folds: int):
        self._data_features = data_features
        self._output_samples = output_samples
        self._cross_validation_folds = cross_validation_folds
        self._n_alphas = n_alphas
        self._model = None
        self._fitted_coefficients: ColumnVector = None

    def fit(self):
        raise NotImplementedError("Any subclass MUST implement this method!")

    def calc_residuals(self):
        return self._output_samples - self._data_features.dot(self._fitted_coefficients)


In [92]:
#@title Generate Preconditioner Sketch Method
%%cython -f
# distutils: extra_compile_args = -fopenmp
# distutils: extra_link_args = -fopenmp
# cython: language_level = 3, boundscheck = False, wraparound = False
# cython: initializedcheck = False, nonecheck = False

cimport openmp
import numpy as np
cimport numpy as np
from cython.parallel cimport prange
from numpy.math cimport INFINITY
from libc.stdlib cimport rand, RAND_MAX, srand, malloc, free, abort
from scipy.fft import dct

cdef inline void _random_sign_change(const double[:, ::1] data_matrix, double[:, ::1] sketched_mat,
                                     const double switch_sign_probability) nogil:
    cdef Py_ssize_t rows_num = data_matrix.shape[0]
    cdef Py_ssize_t column_num = data_matrix.shape[1]
    cdef Py_ssize_t i, j

    for i in range(rows_num):
        if <double>rand() / <double>RAND_MAX <= switch_sign_probability:
            for j in range(column_num):
                sketched_mat[i, j] = -data_matrix[i, j]
        else:
            for j in range(column_num):
                sketched_mat[i, j] = data_matrix[i, j]

cdef inline double[:, ::1] _pick_random_rows(double[:, ::1] sketched_mat, const unsigned int sampled_rows):
    cdef Py_ssize_t* picked_rows = <Py_ssize_t*> malloc(sizeof(Py_ssize_t) * sampled_rows)
    cdef Py_ssize_t[:] picked_rows_view
    cdef Py_ssize_t total_rows_num = sketched_mat.shape[0]
    cdef int row_index
    if picked_rows == NULL:
        abort()

    for row_index in prange(sampled_rows, nogil=True, schedule="static"):
        picked_rows[row_index] = <Py_ssize_t>(rand() % total_rows_num)

    picked_rows_view = <Py_ssize_t[:sampled_rows]> picked_rows
    sketched_mat = sketched_mat.base[picked_rows_view, :]  # Unfortunately, this is a Python object call...
    free(picked_rows)
    return sketched_mat


def generate_sketch_preconditioner(const double[:, ::1] data_matrix, const unsigned int sampled_rows,
                                   double[:, ::1] sketched_mat, const unsigned int seed,
                                   const double switch_sign_probability):
    srand(seed)  # Seeding the random number generator.
    _random_sign_change(data_matrix, sketched_mat, switch_sign_probability)
    sketched_mat = dct(sketched_mat, norm='ortho')
    #return _pick_random_rows(sketched_mat, sampled_rows)  # TODO: Check run time in Google Colab.
    return sketched_mat.base[np.random.randint(low=0, high=sketched_mat.shape[0], size=sampled_rows), :]

print("Loaded Sketch-Preconditioner Method successfully!")


Loaded Sketch-Preconditioner Method successfully!


In [93]:
#@title Basic Caratheodory-Booster in Cython
%%cython -f
# cython: language_level=3, boundscheck=False, wraparound=False
# cython: initializedcheck=False, cdivision=True, nonecheck=False
cimport openmp
import numpy as np
cimport numpy as np
from cython.parallel import prange
from numpy.math cimport INFINITY


def cluster_average(const double[::1, :] cluster, const double[::1] weights):
    cdef Py_ssize_t cluster_size = cluster.shape[1], d = cluster.shape[0], i, j
    cdef double[::1] cluster_mean = np.empty(d)
    cdef double weights_sum = get_vector_sum(weights)

    for i in prange(d, nogil=True, schedule="static"):
        cluster_mean[i] = 0
        for j in range(cluster_size):
            cluster_mean[i] += weights[j] * cluster[i, j]
        cluster_mean[i] /= weights_sum

    return cluster_mean, weights_sum


cdef inline double get_vector_sum(const double[::1] vector) nogil:
    cdef double vector_sum = 0.0
    cdef Py_ssize_t i, n = vector.shape[0]

    for i in prange(n, schedule="static"):
        vector_sum += vector[i]

    return vector_sum

def caratheodory_set_python(np.ndarray[double, ndim=2] points,
                            np.ndarray[double, ndim=1, mode='c'] weights):
    """
    This method computes Caratheodory subset for the columns of a :math:`dxn` Matrix, with given weights.

    Args:
        points(Matrix): A :math:`dxn` Matrix.
        weights(ColumnVector): A weights for the columns of :math:`A`.

    Returns:
        A Caratheodory subset of :math:`d^{2]+1` columns of :math:`A, as a :math:`dx(d^{2}+1)` Matrix,
        and their weights as a ColumnVector.
    """
    cdef Py_ssize_t dim = len(points)  # The dimension of all rows, or rows of the matrix, d.
    cdef Py_ssize_t left_args = points.shape[1], i, j, n = points.shape[1], start_index
    cdef double alpha, temp_div, v_sum
    cdef np.ndarray[double, ndim=2] diff_points
    cdef np.ndarray[double, ndim=1] v
    cdef np.ndarray left_indices = np.arange(0, n, dtype=int)

    while left_args > dim + 1:
        diff_points = points[:, left_indices]
        diff_points = (diff_points[:, 1:].T - diff_points[:, 0].T).T

        v = np.linalg.svd(diff_points, full_matrices=True)[2][left_args - 2]

        v_sum = get_vector_sum(v)
        v = np.insert(v, [0], -v_sum)

        alpha = INFINITY
        for i in range(left_args):
            if v[i] > 0:
                temp_div = weights[left_indices[i]] / v[i]
                if temp_div < alpha:
                    alpha = temp_div

        weights[left_indices] -= alpha * v
        left_indices = np.nonzero(weights > 1e-15)[0]
        left_args = len(left_indices)

    return np.array(weights)[left_indices], left_indices


print("Loaded Cython Caratheodory Set successfully!")


Loaded Cython Caratheodory Set successfully!


In [94]:
#@title Solvers Boosters - Caratheodory and Cholesky boosters
# -*- coding: utf-8 -*-
"""
method_boosters.py - Boosters for least-square solvers
======================================================

This module contains all available boosters for least-square methods, the Cholesky-booster and the Caratheodory-booster.
See the following example of using any booster, i.e cholesky-booster.

Example:
    boosted_solver = cholesky_booster(existing_solver)

"""

import numpy as np
from numpy.linalg import cholesky

_cholesky_covariance: Callable = lambda all_data_mat, clusters_count: cholesky(all_data_mat.T.dot(all_data_mat)).T


@ex.capture
def _LMS_coreset(all_data_mat: Matrix, cross_validation_folds: int, clusters_count: int) -> Matrix:
    sub_matrices: List[Matrix] = np.array_split(all_data_mat, cross_validation_folds)
    coreset: Matrix = np.vstack([create_coreset_fast_caratheodory(sub_mat, clusters_count) for sub_mat in sub_matrices])
    return coreset


def __booster_template(coreset_action: Callable, booster_name: str = "") -> Callable:
    def __booster(existing_regression_method: Callable, perform_normalization: bool = False) -> Callable:

        class _BoostedSolver(BaseSolver):
            @ex.capture
            def __init__(self, data_features: Matrix, output_samples: ColumnVector, n_alphas: int,
                         cross_validation_folds: int, _rnd):
                super(_BoostedSolver, self).__init__(data_features, output_samples, n_alphas, cross_validation_folds)
                self._inner_model = None
                data_length, data_dim = self._data_features.shape
                self._beta: Scalar = self._cross_validation_folds * (data_dim + 1) ** 2 + self._cross_validation_folds
                self._beta = np.sqrt(self._beta / data_length)
                self._perform_normalization = perform_normalization

            def fit(self):
                all_data_mat: Matrix = np.hstack((self._data_features, self._output_samples.reshape(-1, 1)))
                all_data_coreset: Matrix = coreset_action(all_data_mat, self._cross_validation_folds)
                L: Matrix = all_data_coreset[:, :-1]
                new_output_samples: ColumnVector = all_data_coreset[:, -1]

                if self._perform_normalization:
                    L *= self._beta
                    new_output_samples *= self._beta

                self._inner_model = existing_regression_method(L, new_output_samples, self._n_alphas,
                                                               self._cross_validation_folds)

                self._fitted_coefficients = self._inner_model.fit()
                return self._fitted_coefficients

        _BoostedSolver.__name__ = f'{booster_name}{existing_regression_method.__name__}'
        return _BoostedSolver
    return __booster


cholesky_booster: Callable = __booster_template(_cholesky_covariance, booster_name="Cholesky-boosted")
caratheodory_booster: Callable = __booster_template(_LMS_coreset, booster_name="Caratheodory-boosted")


def _greedy_split(arr: Vector, n: int, axis: int = 0, default_block_size: int = 1) -> (List[Vector], Vector):
    """Greedily splits an array into n blocks.

    Splits array arr along axis into n blocks such that:
        - blocks 1 through n-1 are all the same size
        - the sum of all block sizes is equal to arr.shape[axis]
        - the last block is nonempty, and not bigger than the other blocks

    Intuitively, this "greedily" splits the array along the axis by making
    the first blocks as big as possible, then putting the leftovers in the
    last block.
    """
    length: int = arr.shape[axis]

    # compute the size of each of the first n-1 blocks
    if length < n:
        n = default_block_size
    block_size: int = np.floor(length / float(n))

    # the indices at which the splits will occur
    ix: Vector = np.arange(block_size, length, block_size, dtype=int)
    return np.split(arr, ix, axis), ix


def fast_caratheodory_set_python(points: Matrix, weights: ColumnVector, accuracy_time_tradeoff_const: int,
                                 cluster_inner_indices=None) -> (Matrix, ColumnVector):
    """
    This method computes Caratheodory subset for the columns of a :math:`dxn` Matrix, with given weights, using the
    fast Caratheodory subset algorithm with a given number of clusters.

    Args:
        points(Matrix): A :math:`dxn` Matrix.
        weights(ColumnVector): A weights for the columns of :math:`A`.
        accuracy_time_tradeoff_const(int): The number of clusters to use for computing the Caratheodory subset.
        cluster_inner_indices:

    Returns:
        A Caratheodory subset of :math:`d^{2]+1` columns of :math:`A, as a :math:`dx(d^{2}+1)` Matrix,
        and their weights as a ColumnVector.
    """
    points_num: int = points.shape[1]  # The number of points, or columns of the matrix, :math:`n`.
    dim: int = len(points)  # The dimension of all rows, or rows of the matrix, :math:`d`.

    if points_num <= dim + 1:
        returned_indices = cluster_inner_indices
        if cluster_inner_indices is None:
            returned_indices = [x.tolist() for x in np.arange(points_num)]
        return weights, returned_indices

    # Split points into :math:`k` clusters and find the weighted means of every cluster.
    clusters, split_index = _greedy_split(points, accuracy_time_tradeoff_const, axis=1, default_block_size=dim + 2)
    split_weights = np.split(weights, split_index)
    if cluster_inner_indices is None:
        cluster_inner_indices = np.arange(points_num)
    split_indices = np.split(cluster_inner_indices, split_index)
    arguments_to_parallel_mean = list(zip(clusters, split_weights))
    means_and_clusters_weights = compute_serial(cluster_average, arguments_to_parallel_mean)

    clusters_means = np.asfortranarray(np.vstack([result[0] for result in means_and_clusters_weights]))
    clusters_weights = np.array([result[1] for result in means_and_clusters_weights])

    # Perform caratheodory method on the set of means and the total weights of each cluster.
    coreset_weights, chosen_indices = caratheodory_set_python(clusters_means.T, clusters_weights)

    C = []
    c_weights = []
    all_chosen_indices = []
    # Take C as the union of rows from clusters which the caratheodory set consists of.
    for cluster_total_weight, cluster_index in zip(coreset_weights, chosen_indices):
        cluster = clusters[cluster_index]
        cluster_weights = split_weights[cluster_index]
        C.append(cluster)
        all_chosen_indices += np.ravel(split_indices[cluster_index]).tolist()
        c_weights.append(cluster_total_weight * cluster_weights / np.sum(cluster_weights))

    # Assign new weights for each row in C.
    # Perform fast caratheodory method recursively.
    C = np.hstack(C)
    c_weights = np.hstack(c_weights)
    new_weights, all_chosen_indices = fast_caratheodory_set_python(C, c_weights, accuracy_time_tradeoff_const,
                                                                   all_chosen_indices)
    return new_weights, all_chosen_indices


def create_coreset_fast_caratheodory(A: Matrix, clusters_count: int) -> Matrix:
    """
    This method computes the outer-products of the rows of the input matrix. Then it computes a coreset for it,
    using :func:`fast_caratheodory_set_python` with ``clusters_count`` clusters.

    Args:
        A(Matrix): An input :math:nxd matrix.
        clusters_count(int): The number of clusters to use for computing the coreset.

    Returns:
        A :math:`(d^{2}+1)xd` Matrix coreset for the input matrix.
    """
    n: int = A.shape[0]  # The number of points, or rows of the matrix, n.

    rows_outer_products: Matrix = np.einsum("ij,ik->ijk", A, A, optimize=True)
    rows_outer_products = rows_outer_products.reshape((n, -1)).T
    weights, remaining_indices = fast_caratheodory_set_python(rows_outer_products, np.ones(n) / n, clusters_count)
    reduced_mat: Matrix = np.multiply(A[remaining_indices, :].T, np.sqrt(n * weights)).T
    return reduced_mat

print("Solvers Boosters loaded successfully!")


Solvers Boosters loaded successfully!


In [95]:
#@title Linear-Regression Solvers
# -*- coding: utf-8 -*-
"""
linear_regression.py - Linear-Regression solvers module
=======================================================

This module contains all available Linear-Regression solvers.
These solvers can be received by using the only public method :func:`get_method`.

Example:
    get_linear_regression_method(LinearRegressionMethods.SVDBased) - 
      Creating the standard Numpy solver for Linear-Regression.

"""

from numpy.linalg import lstsq, inv, pinv, qr
from scipy.linalg import solve_triangular
from scipy.sparse.linalg import lsqr


class _SVDSolver(BaseSolver):
    @ex.capture
    def __init__(self, data_features: Matrix, output_samples: ColumnVector, cross_validation_folds: int,
                 n_alphas: int = -1):
        r"""
        The standard solver of Numpy for Linear-Regression.

        Args:
            data_features(Matrix): The input data matrix :math:`n \times d`.
            output_samples(ColumnVector): The output for the given inputs, :math:`n \times 1`.
            n_alphas(int): The number of total regularization terms which will be tested by this solver.
            cross_validation_folds(int): The number of cross-validation folds used in this solver.

        """
        super(_SVDSolver, self).__init__(data_features, output_samples, n_alphas, cross_validation_folds)
        self._model = None

    def fit(self) -> ColumnVector:
        """
        The method which fits the requested model to the given data.
        """
        self._fitted_coefficients = lstsq(self._data_features, self._output_samples, rcond=-1)[0]
        return self._fitted_coefficients


class _QRSolver(BaseSolver):
    @ex.capture
    def __init__(self, data_features: Matrix, output_samples: ColumnVector, cross_validation_folds: int,
                 n_alphas: int = -1):
        r"""
        A solver for Linear-Regression, based on QR-decomposition.

        Args:
            data_features(Matrix): The input data matrix :math:`n \times d`.
            output_samples(ColumnVector): The output for the given inputs, :math:`n \times 1`.
            n_alphas(int): The number of total regularization terms which will be tested by this solver.
            cross_validation_folds(int): The number of cross-validation folds used in this solver.

        """
        super(_QRSolver, self).__init__(data_features, output_samples, n_alphas, cross_validation_folds)
        self._model = None

    def fit(self) -> ColumnVector:
        """
        The method which fits the requested model to the given data.
        """
        q, r = qr(self._data_features)
        self._fitted_coefficients = solve_triangular(r, q.T.dot(self._output_samples), lower=False, check_finite=False)
        return self._fitted_coefficients


class _NormalEquationsSolver(BaseSolver):
    @ex.capture
    def __init__(self, data_features: Matrix, output_samples: ColumnVector, cross_validation_folds: int,
                 n_alphas: int = -1):
        r"""
        A solver for Linear-Regression, based on solving the Normal-Equations.

        Args:
            data_features(Matrix): The input data matrix :math:`n \times d`.
            output_samples(ColumnVector): The output for the given inputs, :math:`n \times 1`.
            n_alphas(int): The number of total regularization terms which will be tested by this solver.
            cross_validation_folds(int): The number of cross-validation folds used in this solver.

        """
        super(_NormalEquationsSolver, self).__init__(data_features, output_samples, n_alphas, cross_validation_folds)
        self._model = None

    def fit(self) -> ColumnVector:
        """
        The method which fits the requested model to the given data.
        """
        self._fitted_coefficients = pinv(self._data_features).dot(self._output_samples)
        return self._fitted_coefficients


class _SketchPreconditioerSolver(BaseSolver):
    @ex.capture
    def __init__(self, data_features: Matrix, output_samples: ColumnVector, cross_validation_folds: int, _seed,
                 n_alphas: int = -1):
        r"""
        A solver for Linear-Regression, based on solving the Normal-Equations.

        Args:
            data_features(Matrix): The input data matrix :math:`n \times d`.
            output_samples(ColumnVector): The output for the given inputs, :math:`n \times 1`.
            n_alphas(int): The number of total regularization terms which will be tested by this solver.
            cross_validation_folds(int): The number of cross-validation folds used in this solver.

        """
        super(_SketchPreconditioerSolver, self).__init__(data_features, output_samples, n_alphas,
                                                         cross_validation_folds)
        self._model = None
        self._seed = _seed

    @ex.capture(prefix="sketch_preconditioned_config")
    def fit(self, sampled_rows: float, switch_sign_probability: float, min_sampled_rows: float) -> ColumnVector:
        """
        The method which fits the requested model to the given data.
        """
        num_of_rows: int = max(int(sampled_rows * len(self._data_features)), int(min_sampled_rows))
        _, R = qr(generate_sketch_preconditioner(self._data_features, num_of_rows, np.empty_like(self._data_features),
                                                 self._seed, switch_sign_probability))
        partial_solution: ColumnVector = lsqr(self._data_features.dot(inv(R)), self._output_samples,
                                              atol=1e-15, btol=1e-15)[0]
        self._fitted_coefficients = solve_triangular(R, partial_solution, lower=False, check_finite=False)
        return self._fitted_coefficients


class _SketchInverseSolver(BaseSolver):
    @ex.capture
    def __init__(self, data_features: Matrix, output_samples: ColumnVector, cross_validation_folds: int,
                 n_alphas: int = -1):
        r"""
        A solver for Linear-Regression, based on boosting the algorithm which solves the Normal-Equations,
        using fast Caratheodory method.

        Args:
            data_features(Matrix): The input data matrix :math:`n \times d`.
            output_samples(ColumnVector): The output for the given inputs, :math:`n \times 1`.
            n_alphas(int): The number of total regularization terms which will be tested by this solver.
            cross_validation_folds(int): The number of cross-validation folds used in this solver.

        """
        super(_SketchInverseSolver, self).__init__(data_features, output_samples, -1, cross_validation_folds)
        self._model = None

    @ex.capture
    def fit(self, clusters_count) -> ColumnVector:
        """
        The method which fits the requested model to the given data.
        """
        coreset: Matrix = create_coreset_fast_caratheodory(self._data_features, clusters_count)
        adapted_data, adapted_output, outputs_sum = self._preprocess_data()
        weights, chosen_indices = fast_caratheodory_set_python(adapted_data.T, adapted_output, clusters_count)
        a_times_outputs: ColumnVector = outputs_sum * adapted_data[chosen_indices, :].T.dot(weights)
        self._fitted_coefficients = inv(coreset.T.dot(coreset)).dot(a_times_outputs)
        return self._fitted_coefficients

    def _preprocess_data(self):
        data_copy: Matrix = self._data_features.copy()
        output_copy: ColumnVector = self._output_samples.copy()
        negative_indices: ColumnVector = np.argwhere(self._output_samples < 0)
        data_copy[negative_indices, :] *= -1
        output_copy[negative_indices] *= -1
        output_sum = np.sum(output_copy)
        return data_copy, output_copy/output_sum, output_sum


_sketch_cholesky_linear_regression: Callable = cholesky_booster(_SVDSolver)
_caratheodory_booster_linear_regression: Callable = caratheodory_booster(_SVDSolver, perform_normalization=False)

# A private dictionary used for creating the solvers factory :func:`get_method`.
_linear_regressions_methods: Dict[str, Callable] = {
    LinearRegressionMethods.SVDBased: _SVDSolver,
    LinearRegressionMethods.QRBased: _QRSolver,
    LinearRegressionMethods.NormalEquationsBased: _NormalEquationsSolver,
    LinearRegressionMethods.SketchAndCholesky: _sketch_cholesky_linear_regression,
    LinearRegressionMethods.BoostedSVDSolver: _caratheodory_booster_linear_regression,
    LinearRegressionMethods.SketchAndInverse: _SketchInverseSolver,
    LinearRegressionMethods.SketchPreconditioned: _SketchPreconditioerSolver
}

# A factory which creates the requested linear-regression solvers.
get_linear_regression_method: Callable = create_factory(_linear_regressions_methods, are_methods=True)

print("Linear-Regression Methods loaded successfully!")


Linear-Regression Methods loaded successfully!


In [96]:
#@title Lasso-Regression Solvers
# -*- coding: utf-8 -*-
"""
lasso_regression.py - Lasso-Regression solvers module
=====================================================

This module contains all available Lasso-Regression solvers.
These solvers can be received by using the only public method :func:`get_method`.

Example:
    get_lass_regresion_method(LassoRegressionMethods.SkLearnLassoRegression) - 
      Creating the Scikit-Learn solver for Lasso-Regression.

"""

from sklearn.linear_model import LassoCV


class _SkLearnLassoSolver(BaseSolver):
    @ex.capture
    def __init__(self, data_features: Matrix, output_samples: ColumnVector, n_alphas: int,
                 cross_validation_folds: int, _rnd):
        """
        The standard solver of Scikit-Learn for Lasso-Regression.

        Args:
            data_features(Matrix): The input data matrix ``nxd``.
            output_samples(ColumnVector): The output for the given inputs, ``nx1``.
            n_alphas(int): The number of total regularization terms which will be tested by this solver.
            cross_validation_folds(int): The number of cross-validation folds used in this solver.

        """
        super(_SkLearnLassoSolver, self).__init__(data_features, output_samples, n_alphas, cross_validation_folds)
        self._model = LassoCV(cv=cross_validation_folds, n_alphas=n_alphas, 
                              random_state=_rnd, normalize=False)

    def fit(self):
        """
        The method which fits the requested model to the gicen data.
        """
        self._model.fit(self._data_features, self._output_samples)
        self._fitted_coefficients = self._model.coef_
        return self._fitted_coefficients


_caratheodory_boosted_lasso_regression: Callable = caratheodory_booster(_SkLearnLassoSolver,
                                                                        perform_normalization=True)

# A private dictionary used for creating the solvers factory :func:`get_method`.
_lasso_regressions_methods: Dict[str, Callable] = {
    LassoRegressionMethods.SkLearnLassoRegression: _SkLearnLassoSolver,
    LassoRegressionMethods.BoostedLassoRegression: _caratheodory_boosted_lasso_regression
}

# A factory which creates the requested Lasso-Regression solvers.
get_lasso_regression_method: Callable = create_factory(_lasso_regressions_methods, are_methods=True)
print("Lasso-Regression Methods loaded successfully!")


Lasso-Regression Methods loaded successfully!


In [97]:
#@title Ridge-Regression Solvers
# -*- coding: utf-8 -*-
"""
ridge_regression.py - Ridge-Regression solvers module
=====================================================

This module contains all available Ridge-Regression solvers.
These solvers can be received by using the only public method :func:`get_method`.

Example:
    get_ridge_regreesion_method(RidgeRegressionMethods.SkLearnLassoRegression) - 
      Creating the Scikit-Learn solver for Ridge-Regression.

"""

from sklearn.linear_model import RidgeCV


class _SkLearnRidgeSolver(BaseSolver):
    @ex.capture
    def __init__(self, data_features: Matrix, output_samples: ColumnVector, n_alphas: int,
                 cross_validation_folds: int):
        """
        The standard solver of Scikit-Learn for Lasso-Regression.

        Args:
            data_features(Matrix): The input data matrix ``nxd``.
            output_samples(ColumnVector): The output for the given inputs, ``nx1``.
            n_alphas(int): The number of total regularization terms which will be tested by this solver.
            cross_validation_folds(int): The number of cross-validation folds used in this solver.

        """
        super(_SkLearnRidgeSolver, self).__init__(data_features, output_samples, n_alphas, cross_validation_folds)
        alphas: ColumnVector = np.random.randn(n_alphas)
        self._model = RidgeCV(cv=cross_validation_folds, alphas=alphas, normalize=False)

    def fit(self):
        """
        The method which fits the requested model to the gicen data.
        """
        self._model.fit(self._data_features, self._output_samples)
        self._fitted_coefficients = self._model.coef_
        return self._fitted_coefficients


_caratheodory_boosted_ridge_regression: Callable = caratheodory_booster(_SkLearnRidgeSolver,
                                                                        perform_normalization=False)

# A private dictionary used for creating the solvers factory :func:`get_method`.
_ridge_regressions_methods: Dict[str, Callable] = {
    RidgeRegressionMethods.SkLearnRidgeRegression: _SkLearnRidgeSolver,
    RidgeRegressionMethods.BoostedRidgeRegression: _caratheodory_boosted_ridge_regression
}

# A factory which creates the requested Ridge-Regression solvers.
get_ridge_regression_method: Callable = create_factory(_ridge_regressions_methods, are_methods=True)

print("Ridge-Regression Methods loaded successfully!")


Ridge-Regression Methods loaded successfully!


In [98]:
#@title ElasticNet-Regression Solvers
# -*- coding: utf-8 -*-
"""
elastic_net_regressions.py - ElasticNet-Regression solvers module
=================================================================

This module contains all available ElasticNet-Regression solvers.
These solvers can be received by using the only public method :func:`get_method`.

Example:
    get_relasticnet_regression-method(ElasticNetRegressionMethods.SkLearnLassoRegression) 
      - Creating the Scikit-Learn solver for ElasticNet-Regression.

"""

from sklearn.linear_model import ElasticNetCV


class _SkLearnElasticNetSolver(BaseSolver):
    @ex.capture
    def __init__(self, data_features: Matrix, output_samples: ColumnVector, n_alphas: int,
                 cross_validation_folds: int, elastic_net_factor: Scalar, _rnd):
        """
        The standard solver of Scikit-Learn for Lasso-Regression.

        Args:
            data_features(Matrix): The input data matrix ``nxd``.
            output_samples(ColumnVector): The output for the given inputs, ``nx1``.
            n_alphas(int): The number of total regularization terms which will be tested by this solver.
            cross_validation_folds(int): The number of cross-validation folds used in this solver.

        """
        super(_SkLearnElasticNetSolver, self).__init__(data_features, output_samples, n_alphas, cross_validation_folds)
        self._model = ElasticNetCV(cv=cross_validation_folds, n_alphas=n_alphas, random_state=_rnd,
                                   l1_ratio=elastic_net_factor, normalize=False)

    def fit(self):
        """
        The method which fits the requested model to the gicen data.
        """
        self._model.fit(self._data_features, self._output_samples)
        self._fitted_coefficients = self._model.coef_
        return self._fitted_coefficients

_caratheodory_boosted_elastic_net_regression: Callable = caratheodory_booster(_SkLearnElasticNetSolver,
                                                                              perform_normalization=True)

# A private dictionary used for creating the solvers factory 'get_method'.
_elastic_net_regressions_methods: Dict[str, Callable] = {
    ElasticNetRegressionMethods.SkLearnElasticNetRegression: _SkLearnElasticNetSolver,
    ElasticNetRegressionMethods.BoostedElasticNetRegression: _caratheodory_boosted_elastic_net_regression
}


# A factory which creates the relevant Elastic-Net-Regression solver.
get_elasticnet_regression_method: Callable = create_factory(_elastic_net_regressions_methods, are_methods=True)

print("ElasticNet-Regression Methods loaded successfully!")


ElasticNet-Regression Methods loaded successfully!


In [99]:
#@title __init__
# -*- coding: utf-8 -*-
"""
__init__.py - A solvers' factory method
========================================

This module contains the factory method :func:`get_method` which creates the requested solvers for an experiment.
See the following examples on creating solvers.

Examples:
    get_methods(AlgorithmsType.LinearRegression, []) - Creates all available solvers for Linear-Regression.
    get_methods(AlgorithmsType.LinearRegression, [LinearRegressionMethods.SVDBased]) - Creates only the standard Numpy
     solver for Linear-Regression.

"""

# A private dictionary used for get_method.
_algorithms_type_to_algorithms: Dict = {
    AlgorithmsType.LinearRegression: (LinearRegressionMethods, get_linear_regression_method),
    AlgorithmsType.RidgeRegression: (RidgeRegressionMethods, get_ridge_regression_method),
    AlgorithmsType.LassoRegression: (LassoRegressionMethods, get_lasso_regression_method),
    AlgorithmsType.ElasticNetRegression: (ElasticNetRegressionMethods, get_elasticnet_regression_method)
}


def get_methods(requested_algorithms_type: AlgorithmsType, compared_methods: List) -> List:
    """
    A factory which creates the requested solvers.

    Args:
        requested_algorithms_type(AlgorithmsType): The name of the solvers type, i.e linear-regression
        compared_methods(List): A list of specific solvers to create. If empty, all solvers of a given type are created.

    Returns:
         A list of all relevant solvers.

    """
    solvers_names_list, solvers_factory = _algorithms_type_to_algorithms[requested_algorithms_type]
    solvers: List = list()
    if is_empty(compared_methods):
        for solver_name in solvers_names_list:
            solvers.append(solvers_factory(solver_name))
    else:
        for solver_name in compared_methods:
            solvers.append(solvers_factory(solver_name))
    return solvers

print("ComparedAlgorithms.__init__ loaded successfully!")


ComparedAlgorithms.__init__ loaded successfully!


#Main Components

In [100]:
#@title Database loader
# -*- coding: utf-8 -*-
"""
database_loader.py - The data management module
===============================================

This module handles the fetching of the data from the local resources path, given in the configuration and arranging it
for our purposes of estimations. See the example for fetching the 3D Road Network database.

Example:
    get_data(DatabaseType.ThreeDRoadNetwork) - Creating the standard Numpy solver for Linear-Regression.

"""

from numpy.random import randn
from sklearn.preprocessing import scale


@ex.capture(prefix="synthetic_data_config")
def _get_synthetic_data(data_size: int, features_num: int) -> (Matrix, ColumnVector):
    """
    A method which creates a random matrix of ``size data_size x features_num`` and a ``random 1 x data_size``
    random vector.

    Args:
        data_size(int): The number of samples in the data - ``n``.
        features_num(int): The number of columns in the data - ``d``.

    Returns:
        A random ``size data_size x features_num`` Matrix and a random ``1 x data_size`` ColumnVector.

    """
    return randn(data_size, features_num), randn(data_size)


@ex.capture
def _get_3d_road_network_data(resources_path: str) -> (Matrix, ColumnVector):
    """
    A method which loads the 3D Road Network database from the given local path.
    See https://archive.ics.uci.edu/ml/datasets/3D+Road+Network+(North+Jutland,+Denmark)

    Args:
        resources_path(str): The path in which this database is stored in.

    Returns:
        A Matrix of the features, besides the height, and a ColumnVector of the height feature.

    """
    data: Matrix = scale(np.loadtxt(os.path.join(resources_path, "3D_spatial_network.txt"), delimiter=','))
    output_samples: ColumnVector = data[:, -1]
    data_matrix: Matrix = np.ascontiguousarray(data[:, 1:3])
    return data_matrix, output_samples


@ex.capture
def _get_house_sales_in_king_county_data(resources_path: str) -> (Matrix, ColumnVector):
    """
    A method which loads the House sales in King County database from the given local path.
    See https://www.kaggle.com/harlfoxem/housesalesprediction

    Args:
        resources_path(str): The path in which this database is stored in.

    Returns:
        A Matrix of the features, besides the price, and a ColumnVector of the price feature.

    """
    df = pd.read_csv(os.path.join(resources_path, "kc_house_data.csv"))
    output_samples: ColumnVector = scale(df["price"].to_numpy())
    data_matrix: Matrix = np.ascontiguousarray(scale(df[["bedrooms", "sqft_living", "sqft_lot", "floors", "waterfront",
                                                         "sqft_above", "sqft_basement", "yr_built"]].to_numpy()))
    return data_matrix, output_samples


@ex.capture
def _get_household_electric_power_consumption_data(resources_path: str) -> (Matrix, ColumnVector):
    """
    A method which loads the Household electric power consumption database from the given local path.
    See https://archive.ics.uci.edu/ml/datasets/Individual+household+electric+power+consumption.

    Args:
        resources_path(str): The path in which this database is stored in.

    Returns:
        A Matrix of the features, besides the voltage, and a ColumnVector of the voltage feature.

    """
    df = pd.read_csv(os.path.join(resources_path, "household_power_consumption.txt"), sep=';', na_values="?")
    df.dropna(axis=0, inplace=True)
    output_samples: ColumnVector = scale(pd.to_numeric(df["Voltage"]).to_numpy())
    data_matrix: Matrix = np.ascontiguousarray(scale(df[["Global_active_power", "Global_reactive_power"]].astype("float64").to_numpy()))
    return data_matrix, output_samples


# A private dictionary used to create the method "get_data"
_database_type_to_function: Dict[str, Callable] = {
    DatabaseType.Synthetic: _get_synthetic_data,
    DatabaseType.ThreeDRoadNetwork: _get_3d_road_network_data,
    DatabaseType.HouseSalesInKingCounty: _get_house_sales_in_king_county_data,
    DatabaseType.IndividualHouseholdElectricPowerConsumption: _get_household_electric_power_consumption_data
}

# The public method which fetches the data loading methods.
get_data: Callable = create_factory(_database_type_to_function)

print("Database loader loaded successfully!")


Database loader loaded successfully!


In [101]:
#@title Experiments
from numpy.linalg import norm

@ex.capture()
def _run_time_experiment(compared_solvers: List, data_matrix: Matrix, output_samples: Vector,
                         run_time_experiments_config):
    all_logs: Dict = dict()
    run_time_compared_data_sizes: List = run_time_experiments_config["run_time_compared_data_sizes"]
    calc_transpose_dot_residuals: bool = run_time_experiments_config["calc_transpose_dot_residuals"]

    for solver in compared_solvers:
        error_raised: bool = False
        log_fields: List = [LogFields.DataSize, LogFields.Coefficients, LogFields.Residuals,
                            LogFields.DurationInSeconds]
        if calc_transpose_dot_residuals:
            log_fields.append(LogFields.AtTimesErrors)

        data_log = DataLog(log_fields)
        coefficients_list = list()
        residuals_list = list()
        transpose_errors_list = list()
        durations_list = list()

        for data_size in run_time_compared_data_sizes:
            # try:
            partial_data: Matrix = data_matrix[:data_size, :]
            partial_samples: Vector = output_samples[:data_size]
            solver_obj = solver(partial_data, partial_samples)
            coefficients, duration = measure_time(solver_obj.fit)()
            residuals: Vector = solver_obj.calc_residuals()

            coefficients_list.append(coefficients.tolist())
            residuals_list.append(norm(residuals))
            if calc_transpose_dot_residuals:
                transpose_errors_list.append(norm(partial_data.T.dot(residuals)))
            durations_list.append(duration)
            print(f'solver name={solver.__name__}, data size={data_size}, duration={duration}')

            # except IOError:
            #     error_raised = True
            #     continue

        if not error_raised:
            data_log.append(LogFields.DataSize, list(run_time_compared_data_sizes))
            data_log.append(LogFields.Residuals, residuals_list)
            if calc_transpose_dot_residuals:
                data_log.append(LogFields.AtTimesErrors, transpose_errors_list)
            data_log.append(LogFields.Coefficients, coefficients_list)
            data_log.append(LogFields.DurationInSeconds, durations_list)
            all_logs[solver.__name__] = data_log

    return all_logs


@ex.capture(prefix="number_of_alphas_experiments_config")
def _number_of_alphas_experiment(compared_solvers: List, data_matrix: Matrix, output_samples: Vector,
                                 alphas_range: List):
    all_logs: Dict = dict()
    for solver in compared_solvers:
        error_raised: bool = False
        data_log = DataLog([LogFields.Coefficients, LogFields.Residuals, LogFields.DurationInSeconds,
                            LogFields.AlphasCount])
        coefficients_list = list()
        residuals_list = list()
        durations_list = list()

        for current_alphas in alphas_range:
            # try:
              solver_obj = solver(data_matrix, output_samples, n_alphas=current_alphas)
              coefficients, duration = measure_time(solver_obj.fit)()
              residuals: Vector = solver_obj.calc_residuals()

              coefficients_list.append(coefficients.tolist())
              residuals_list.append(norm(residuals))
              durations_list.append(duration)
              print(f'solver name={solver.__name__}, total alphas={current_alphas}, duration={duration}')

            # except IOError:
            #     error_raised = True
            #     continue

        if not error_raised:
            data_log.append(LogFields.AlphasCount, list(alphas_range))
            data_log.append(LogFields.Residuals, residuals_list)
            data_log.append(LogFields.Coefficients, coefficients_list)
            data_log.append(LogFields.DurationInSeconds, durations_list)
            all_logs[solver.__name__] = data_log

    return all_logs


_experiment_type_to_method: Dict = {
    ExperimentType.RunTimeExperiment: _run_time_experiment,
    ExperimentType.NumberOfAlphasExperiment: _number_of_alphas_experiment
}

create_experiment: Callable = create_factory(_experiment_type_to_method, are_methods=True)

print("Experiments loaded successfully!")


Experiments loaded successfully!


In [107]:
#@title Main
# -*- coding: utf-8 -*-
"""
main.py - The main module of the project
========================================

This module contains the config for the experiment in the "config" function.
Running this module invokes the :func:`main` function, which then performs the experiment and saves its results
to the configured results folder. Example for running an experiment: ``python main.py``

"""

def _choose_clusters_num(database_type: str, synthetic_data_dim: int) -> int:
    """
    This method determines the number of clusters for coreset computation, using the number of features in the data.
    If the data has :math:`d` features then the number of used clusters is :math:`2(d+1)^{2}+2`.
    """
    data_dim: int = 1
    if database_type == DatabaseType.Synthetic:
        data_dim = synthetic_data_dim
    elif database_type in [DatabaseType.ThreeDRoadNetwork, 
                           DatabaseType.IndividualHouseholdElectricPowerConsumption]:
        data_dim = 2
    elif database_type == DatabaseType.HouseSalesInKingCounty:
        data_dim = 8
    return 2 * (data_dim + 1) ** 2 + 2


@ex.config
def config():
    """ Config section

    This function contains all possible configuration for all experiments. Full details on each configuration values
    can be found in :mod:`enums.py`.
    """
    compared_algorithms_type: AlgorithmsType = AlgorithmsType.LinearRegression
    compared_methods: List = [LinearRegressionMethods.SketchAndInverse]  # Leave empty for using all solvers.
    numpy_distribution: NumpyDistribution = NumpyDistribution.IntelDistribution
    used_database: DatabaseType = DatabaseType.HouseSalesInKingCounty
    experiment_type: ExperimentType = ExperimentType.RunTimeExperiment
    cross_validation_folds: int = 1
    
    n_alphas: int = 100

    run_time_experiments_config: Dict[str, range] = {
        "run_time_compared_data_sizes": range(500, 22000, 500),
        "calc_transpose_dot_residuals": compared_algorithms_type == AlgorithmsType.LinearRegression
    }
    number_of_alphas_experiments_config: Dict[str, range] = {
        "alphas_range": range(1, 221, 20)
    }

    synthetic_data_config: Dict[str, int] = {
        "data_size": 80000000,
        "features_num": 7
    }
    sketch_preconditioned_config: Dict[str, float] = {
        "sampled_rows": 0.005,
        "switch_sign_probability": 0.5,
        "min_sampled_rows": 100.0
    }
    resources_path: str = r'Resources'
    results_path: str = r'Results'
    clusters_count: int = _choose_clusters_num(used_database, synthetic_data_config["features_num"])
    elastic_net_factor: float = 0.5  # Rho factor in Elastic-Net regularization.


@ex.main
def run_experiment(compared_algorithms_type: AlgorithmsType, compared_methods: List, used_database: DatabaseType,
                   experiment_type: ExperimentType, results_path: str) -> Dict[str, DataLog]:
    """ The main function of this project

    This functions performs the desired experiment according to the given configuration.
    The function then saves all the experiment results to a csv file in the results folder (given in the configuration).
    """
    #sys.setrecursionlimit(1500)
    compared_solvers: List = get_methods(compared_algorithms_type, compared_methods)
    data_matrix, output_samples = get_data(used_database)
    experiment = create_experiment(experiment_type)
    all_logs: Dict[str, DataLog] = experiment(compared_solvers, data_matrix, output_samples)

    for log_name, log in all_logs.items():
        log.save_log(log_name, results_path)

    return all_logs

ex.run()
print("Experiment performed Successfully!")


WARNING - Initializing project - No observers have been added to this run
INFO - Initializing project - Running command 'run_experiment'
INFO - Initializing project - Started


solver name=_SketchInverseSolver, data size=500, duration=0.7006971379996685
solver name=_SketchInverseSolver, data size=1000, duration=0.8450672200015106
solver name=_SketchInverseSolver, data size=1500, duration=1.071158833001391
solver name=_SketchInverseSolver, data size=2000, duration=1.4278948429964657
solver name=_SketchInverseSolver, data size=2500, duration=1.8140475059990422
solver name=_SketchInverseSolver, data size=3000, duration=1.236588383999333
solver name=_SketchInverseSolver, data size=3500, duration=1.3277555169988773
solver name=_SketchInverseSolver, data size=4000, duration=1.4726600840003812
solver name=_SketchInverseSolver, data size=4500, duration=1.5841118909993384
solver name=_SketchInverseSolver, data size=5000, duration=1.8408135739991849
solver name=_SketchInverseSolver, data size=5500, duration=2.1761056650029786
solver name=_SketchInverseSolver, data size=6000, duration=2.2923864020012843
solver name=_SketchInverseSolver, data size=6500, duration=2.180378

INFO - Initializing project - Result: {'_SketchInverseSolver': <__main__.DataLog object at 0x7fdc318b2fd0>}
INFO - Initializing project - Completed after 0:01:18


solver name=_SketchInverseSolver, data size=21500, duration=1.947237046999362
Experiment performed Successfully!


#Unit-Tests

In [103]:
#@title Testing Caratheodory sets methods
import unittest

class TestCaratheodorySet(unittest.TestCase):
    """ A class which contains tests for Caratheodory subset computation."""

    def test_python_Caratheodory_set(self):
        """
        This method tests the basic algorithm for Caratheodory subset algorithm.
        """
        d: int = 15
        n: int = 100
        data: Matrix = np.random.rand(d, n)
        weights: Vector = np.random.random(n)
        weights /= weights.sum()
        epsilon: Scalar = np.finfo(np.float).eps
        new_weights, chosen_indices = caratheodory_set_python(data, weights)
        self.assertAlmostEqual(np.sum(weights), 1, delta=5 * epsilon)
        self.assertTrue(np.all(weights >= -epsilon), f'\nSome negative weights: {weights[np.invert(weights >= 0)]}')
        self.assertLessEqual(len(chosen_indices), d + 1)
        self.assertTrue(np.allclose(data.dot(weights), data[:, chosen_indices].dot(new_weights), atol=1e-10, rtol=0))

    def test_fast_python_Caratheodory_set(self):
        """
        This method tests the faster algorithm for Caratheodory subset algorithm.
        """
        d: int = 3
        n: int = 50
        clusters_count: int = 2 * (d + 1) ** 2 + 2
        data: Matrix = np.asfortranarray(np.random.rand(d, n))
        weights: Vector = np.random.random(n)
        weights /= weights.sum()
        epsilon: Scalar = np.finfo(np.float).eps
        new_weights, chosen_indices = fast_caratheodory_set_python(data, weights, clusters_count)
        self.assertAlmostEqual(np.sum(weights), 1, delta=5 * epsilon)
        self.assertTrue(np.all(weights >= -epsilon), f'\nSome negative weights: {weights[np.invert(weights >= 0)]}')
        self.assertLessEqual(len(chosen_indices), d + 1)
        self.assertTrue(np.allclose(data.dot(weights), data[:, chosen_indices].dot(new_weights), atol=1e-10, rtol=0))
      
print("Caratheodory Set Unit-Tests loaded successfully!")


Caratheodory Set Unit-Tests loaded successfully!


In [104]:
#@title Testing Matrix Coresets Method

class TestMatrixCoreset(unittest.TestCase):
    """ A class which contains tests for methods for coreset computation."""

    def test_outer_products_decomposition(self):
        """
        This test verifies that this `einsum` function computes the outer products of all the matrix's rows.
        This means the sum of these outer products must be equal to the original matrix.
        """
        n: int = 7
        d: int = 2
        A: Matrix = 500 * np.random.randn(n, d)
        rows_outer_products: Matrix = np.einsum("ij,ik->ijk", A, A, optimize=True)
        self.assertTrue(np.allclose(A.T.dot(A), rows_outer_products.sum(axis=0), atol=1e-8, rtol=0))

    def test_python_fast_matrix_coresets(self):
        """
        This test verifies the 'coreset' of the matrix has the same Gram matrix as the original matrix, i.e
        :math:`L^{T}L=A^{T}A`.
        """
        n: int = 500
        d: int = 7
        clusters_count: int = 2 * (d + 1) ** 2 + 2
        A: Matrix = 500 * np.random.randn(n, d)
        reduced_mat: Matrix = create_coreset_fast_caratheodory(A, clusters_count)
        self.assertTrue(np.allclose(reduced_mat.T.dot(reduced_mat), A.T.dot(A), atol=1e-6, rtol=0))
        
print("Coreset Unit-Tests loaded successfully!")


Coreset Unit-Tests loaded successfully!


In [105]:
#@title Run all Unit-Tests
unittest.main(argv=[''], verbosity=2, exit=False)


test_fast_python_Caratheodory_set (__main__.TestCaratheodorySet) ... ok
test_python_Caratheodory_set (__main__.TestCaratheodorySet) ... ok
test_outer_products_decomposition (__main__.TestMatrixCoreset) ... ok
test_python_fast_matrix_coresets (__main__.TestMatrixCoreset) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.488s

OK
